In [1]:
import csv
import xml.etree.ElementTree as ET
import pandas as pd

In [2]:
class JspritSolution(object):
    def __init__(self, cost, routes=None, unassigned=None):
        self.cost = cost
        self.routes = routes
        self.unassigned = unassigned
        self.modified_route = None

In [3]:
class ActType(object):
    PICK_UP = 0
    DROP_OFF = 1
    DELIVERY = 2
    DRIVE = 3
    WAIT = 4
    RETURN = 5
    IDLE = 6

    def __init__(self, type_=None):
        self.type = type_

    @staticmethod
    def get_type_from_string(act_string):
        return {'pickupShipment': ActType.PICK_UP,
                'deliverShipment': ActType.DROP_OFF,
                'delivery': ActType.DELIVERY
                }[act_string]

    @staticmethod
    def get_string_from_type(act_type):
        return {ActType.PICK_UP: 'pickupShipment',
                ActType.DROP_OFF: 'deliverShipment',
                ActType.DELIVERY: 'delivery',
                ActType.RETURN: 'return',
                ActType.WAIT: 'wait',
                ActType.DRIVE: 'drive',
                ActType.IDLE: 'idle',
                }[act_type]

    def __str__(self):
        return self.get_string_from_type(self.type)

    def __repr__(self):
        return self.__str__()

In [4]:
class JspritAct(ActType):

    def __init__(self, type_=None, person_id=None, end_time=None, arrival_time=None):
        super(JspritAct, self).__init__(type_=type_)
        self.person_id = person_id
        self.end_time = end_time
        self.arrival_time = arrival_time

    def get_duration(self):
        return self.end_time - self.arrival_time

    def __str__(self):
        return 'Person{}, end_time {}, arrival_time {}' \
            .format(self.person_id, self.end_time, self.arrival_time)

    def __repr__(self):
        return self.__str__()

In [5]:
class JspritRoute(object):
    acts = None  # type: List[JspritAct]

    def __init__(self, vehicle_id=None, start_time=None, end_time=None, acts=None):
        self.vehicle_id = vehicle_id
        self.start_time = start_time
        self.end_time = end_time
        self.acts = acts

In [6]:
def read_vrp_solution(file_name):
    """Reads solution from output XML file of jsprit
    :return:  JspritSolution
    """
    tree = ET.parse(file_name)
    root = tree.getroot()
    namespace = {'xmlns': 'http://www.w3schools.com'}
    solutions_element = root.find('xmlns:solutions', namespace)
    solution_element = solutions_element.find('xmlns:solution', namespace)
    routes = []
    routes_element = solution_element.find('xmlns:routes', namespace)
    # Theoretically there could be situation when no traveler can be routed
    if routes_element is None:
        return None
    for route_element in routes_element.findall('xmlns:route', namespace):
        acts = []
        for act_element in route_element.findall('xmlns:act', namespace):
            person_id_element = act_element.find('xmlns:shipmentId', namespace)
            if person_id_element is None:
                person_id_element = act_element.find('xmlns:serviceId', namespace)
            act = JspritAct(type_=JspritAct.get_type_from_string(act_element.attrib.get('type')),
                            person_id=int(person_id_element.text),
                            end_time=float(act_element.find('xmlns:endTime', namespace).text),
                            arrival_time=float(act_element.find('xmlns:arrTime', namespace).text)
                            )
            acts.append(act)
        route = JspritRoute(vehicle_id=int(route_element.find('xmlns:vehicleId', namespace).text),
                            start_time=float(route_element.find('xmlns:start', namespace).text),
                            end_time=float(route_element.find('xmlns:end', namespace).text),
                            acts=acts
                            )
        routes.append(route)
    unassigned_jobs_elements = solution_element.findall('xmlns:unassignedJobs', namespace)
    unassigned_job_ids = []
    # there could be unroutable or undeliverable requests
    if unassigned_jobs_elements is not None and len(unassigned_jobs_elements) > 0:
        for unassigned_jobs_element in unassigned_jobs_elements[0].findall('xmlns:job', namespace):
            unassigned_job_ids.append(int(unassigned_jobs_element.attrib.get('id')))

    solution = JspritSolution(cost=float(solution_element.find('xmlns:cost', namespace).text),
                              routes=routes,
                              unassigned=unassigned_job_ids)
    return solution

In [7]:
def read_vrp_initial_solution(file_name):
    """Reads solution from output XML file of jsprit
    :return:  JspritSolution
    """
    tree = ET.parse(file_name)
    root = tree.getroot()
    namespace = {'xmlns': 'http://www.w3schools.com'}
    routes = []
    for route_element in root.findall('xmlns:initialRoutes/xmlns:route', namespace):
        acts = []
        for act_element in route_element.findall('xmlns:act', namespace):
            person_id_element = act_element.find('xmlns:shipmentId', namespace)
            if person_id_element is None:
                person_id_element = act_element.find('xmlns:serviceId', namespace)
            act = JspritAct(type_=JspritAct.get_type_from_string(act_element.attrib.get('type')),
                            person_id=int(person_id_element.text),
                            end_time=float(act_element.find('xmlns:endTime', namespace).text),
                            arrival_time=float(act_element.find('xmlns:arrTime', namespace).text)
                            )
            acts.append(act)
        route = JspritRoute(vehicle_id=int(route_element.find('xmlns:vehicleId', namespace).text),
                            start_time=float(route_element.find('xmlns:start', namespace).text),
                            end_time=float(route_element.find('xmlns:end', namespace).text),
                            acts=acts
                            )
        routes.append(route)

    solution = JspritSolution(cost=0,
                              routes=routes,
                              unassigned=[])
    return solution

In [8]:
file = 'data/wait_cost_1203/solution_sjobo_integrated_STS_all_general_all_BIFR.xml'

In [9]:
tdm = pd.read_csv('data/tdm_sjobo_integrated_STS_all_general_all.csv', header=None)

In [10]:
solution = read_vrp_solution(file)
# solution = read_vrp_initial_solution(file)

In [11]:
tdm = tdm.set_index([0,1])
tdm.columns = ['time', 'distance']

In [12]:
tdm

time  distance
0    1                     
0    0        0.0       0.0
     1     2862.4   57530.0
     2     4443.8  101104.7
     3     5350.5  122468.1
     4     5808.1  125854.7
...           ...       ...
9670 9666  1953.2   30651.9
     9667  2611.8   38451.7
     9668  1927.1   30486.5
     9669  3430.8   71024.4
     9670     0.0       0.0

[93528241 rows x 2 columns]

In [13]:
class shipment(object):
    def __init__(self, person_id=None, pickup_id=None, delivery_id=None):
        self.person_id = person_id
        self.pickup_id = pickup_id
        self.delivery_id = delivery_id

class vehicle(object):
    def __init__(self, vehicle_id=None, start_id=None, end_id=None):
        self.vehicle_id = vehicle_id
        self.start_id = start_id
        self.end_id = end_id

In [14]:
tree = ET.parse(file)
root = tree.getroot()
namespace = {'xmlns': 'http://www.w3schools.com'}
shipments_element = root.find('xmlns:shipments', namespace)
shipment_elements = shipments_element.findall('xmlns:shipment', namespace)
shipments = []
for shipment_element in shipment_elements:
    ind = int(shipment_element.attrib.get('id'))
    pickup_id = int(shipment_element.find('xmlns:pickup', namespace).find('xmlns:location', namespace).find('xmlns:id', namespace).text)
    delivery_id = int(shipment_element.find('xmlns:delivery', namespace).find('xmlns:location', namespace).find('xmlns:id', namespace).text)
    shipments.append(shipment(ind, pickup_id, delivery_id))

In [15]:
tree = ET.parse(file)
root = tree.getroot()
namespace = {'xmlns': 'http://www.w3schools.com'}
vehicles_element = root.find('xmlns:vehicles', namespace)
vehicle_elements = vehicles_element.findall('xmlns:vehicle', namespace)
vehicles = []
for vehicle_element in vehicle_elements:
    ind = int(vehicle_element.find('xmlns:id', namespace).text)
    start_id = int(vehicle_element.find('xmlns:startLocation', namespace).find('xmlns:id', namespace).text)
    end_id = int(vehicle_element.find('xmlns:endLocation', namespace).find('xmlns:id', namespace).text)
    vehicles.append(vehicle(ind, start_id, end_id))

In [16]:
active_time = []
drive_time = []
drive_distance = []
passengers = []
direct_distance = []
direct_time = []
route_time = []
for route in solution.routes:
    route_time.append(route.end_time - route.start_time)
    active_time.append(route.acts[-1].end_time - route.acts[0].end_time + route.acts[0].arrival_time - route.start_time)
    VKT = 0
    npass = 0
    dtime = 0
    ddistance = 0
    points = []
    
#   depot -> first location
    vehicle = [v for v in vehicles if v.vehicle_id == route.vehicle_id][0]
    points.append(vehicle.start_id)
    
#   main route
    for act in route.acts:
        shipment = [s for s in shipments if s.person_id == act.person_id][0]
        if act.type == 0:
            loc_id = shipment.pickup_id
        elif act.type == 1:
            loc_id = shipment.delivery_id 
            npass += 1
            direct = tdm.loc[(shipment.pickup_id, shipment.delivery_id)]
            dtime += direct.time
            ddistance += direct.distance
        else:
            print('should not be here!')
            break
        
        points.append(loc_id)
        
    
            
#   last location -> depot
    points.append(vehicle.end_id)
    
#   save for DataFrame
    td = sum([tdm.loc[(s,e)] for s,e in zip(points, points[1:])])
    drive_time.append(td.time)
    drive_distance.append(td.distance)
    passengers.append(npass)
    direct_distance.append(ddistance)
    direct_time.append(dtime)
    
routes_parsed = pd.DataFrame({
    'drive_time': drive_time,
    'drive_distance': drive_distance,
    'active_time': active_time,
    'passengers': passengers,
    'direct_distance': direct_distance,
    'direct_time': direct_time,
    'route_time': route_time,
    })

In [17]:
routes_parsed

,drive_time,drive_distance,active_time,passengers,direct_distance,direct_time,route_time
0,61830.2,974044.8,65411.5,32,611908.2,38978.8,86264.0
1,27477.1,437274.1,29119.1,16,354619.5,20070.8,55697.4
2,33467.4,530215.6,36263.9,18,366296.7,22830.4,54729.7
3,62252.2,911962.6,67068.8,36,766904.5,49701.4,84788.4
4,38633.1,562110.6,41922.9,23,507831.6,30216.0,65149.7
...,...,...,...,...,...,...,...
653,9381.0,153256.4,10137.7,5,33580.4,2711.1,36595.1
654,5154.8,77678.2,5278.7,3,41290.2,2582.8,34593.7
655,7898.9,135200.2,8316.9,4,70950.5,4234.0,34320.9
656,8418.7,131474.2,8275.4,4,91347.7,5694.8,38399.4


In [18]:
routes_parsed.describe()

,drive_time,drive_distance,active_time,passengers,direct_distance,direct_time,route_time
count,658.000000,6.580000e+02,658.000000,658.000000,6.580000e+02,658.000000,658.000000
mean,33775.788602,5.094375e+05,38848.403343,17.770517,3.752183e+05,23614.997872,60316.104103
std,16534.231827,2.485411e+05,19351.308177,9.524969,2.126143e+05,13053.294216,15747.056012
min,467.600000,4.194700e+03,318.600000,1.000000,1.988700e+03,222.000000,30431.000000
25%,19617.450000,2.937160e+05,23297.600000,9.000000,1.962359e+05,12407.750000,48612.175000
50%,35438.700000,5.322224e+05,39940.550000,18.000000,3.808155e+05,24211.100000,62767.850000
75%,46275.075000,6.854277e+05,52478.975000,25.000000,5.205208e+05,33185.450000,71805.850000
max,72890.600000,1.072414e+06,86225.200000,46.000000,1.036819e+06,62875.600000,86396.800000


In [19]:
routes_parsed.passengers.sum()

11693

In [20]:
routes_parsed.drive_distance.sum()/len(routes_parsed.index)

509437.53996960487

In [21]:
routes_parsed.passengers.sum()

11693

In [22]:
len(solution.unassigned)

4

In [23]:
routes_parsed.sum()

drive_time          22224468.9
drive_distance     335209901.3
active_time         25562249.4
passengers             11693.0
direct_distance    246893661.3
direct_time         15538668.6
route_time          39687996.5
dtype: float64

In [24]:
routes_parsed.drive_distance.sum()/1000

335209.9013

In [25]:
routes_parsed.direct_distance.sum()/1000

246893.66129999998

In [26]:
routes_parsed.direct_time.sum()/60

258977.81000000003

In [27]:
routes_parsed.drive_time.sum()/60

370407.815

In [28]:
routes_parsed.drive_time.sum()/60/60

6173.463583333333

In [29]:
routes_parsed.active_time.sum()/60/60

7100.624833333334

In [30]:
routes_parsed.route_time.sum()/60/60

11024.443472222221

In [31]:
solution.cost

72000005841689.28